In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import undetected_chromedriver as uc
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
import nltk, urllib, random, requests, csv, os, sys, time, datefinder, re
from selenium.webdriver.chrome.options import Options
from openpyxl import Workbook
from openpyxl.styles import Font, Color, Alignment, Border, Side
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.utils import get_column_letter
from tkinter import simpledialog
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from tkinter import simpledialog
from datetime import date
import tkinter as tk
import chromedriver_autoinstaller
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

today = date.today()


class App():
    def __init__(self, master):
        self.master = master
        self.type_integration = None
        self.typeChoice = tk.StringVar()
        self.typeChoice.set(None) 
        self.typeFrame = tk.Frame(master)
        OPTIONS = [('Robertson', 'Robertson'),('Williamson','Williamson'),('Wilson','Wilson'),('Rutherford','Rutherford')]
        for text, value in OPTIONS:
            tk.Radiobutton(self.typeFrame, text=text, variable=self.typeChoice, value=value).pack()
        tk.Button(self.typeFrame, text="Select", command=self.exit).pack()
        self.typeFrame.pack()
    def exit(self):
        self.type_integration = self.typeChoice.get()
        self.master.destroy() 
    def getinput(self):
        return self.type_integration

master = tk.Tk()
app = App(master)
tk.mainloop()
county=(app.getinput()).strip()

audioToTextDelay = 10
delayTime = 2
audioFile = "\\payload.mp3"
URL = "https://www.google.com/recaptcha/api2/demo"
SpeechToTextURL = "https://speech-to-text-demo.ng.bluemix.net/"


def delay():
    time.sleep(random.randint(2, 3))

def audioToText(audioFile):
    driver.execute_script('''window.open("","_blank")''')
    driver.switch_to.window(driver.window_handles[1])
    driver.get(SpeechToTextURL)

    delay()
    audioInput = driver.find_element(By.XPATH, '//*[@id="root"]/div/input')
    audioInput.send_keys(audioFile)

    time.sleep(audioToTextDelay)

    text = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[7]/div/div/div/span')
    while text is None:
        text = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[7]/div/div/div/span')

    result = text.text

    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    return result
def captch_solve():
    iframes = driver.find_elements(By.TAG_NAME, 'iframe')
    audioBtnFound = False
    audioBtnIndex = -1

    for index in range(len(iframes)):
        driver.switch_to.default_content()
        iframe = driver.find_elements(By.TAG_NAME, 'iframe')[index]
        driver.switch_to.frame(iframe)
        driver.implicitly_wait(delayTime)
        try:
            audioBtn = driver.find_element(By.ID, "recaptcha-audio-button")
            audioBtn.click()
            audioBtnFound = True
            audioBtnIndex = index
            break
        except Exception as e:
            pass

    if audioBtnFound:
        try:
            while True:
                # get the mp3 audio file
                src = driver.find_element(By.ID, "audio-source").get_attribute("src")
                print("[INFO] Audio src: %s" % src)

                # download the mp3 audio file from the source
                urllib.request.urlretrieve(src, os.getcwd() + audioFile)

                # Speech To Text Conversion
                key = audioToText(os.getcwd() + audioFile)
                print("[INFO] Recaptcha Key: %s" % key)

                driver.switch_to.default_content()
                iframe = driver.find_elements(By.TAG_NAME, 'iframe')[audioBtnIndex]
                driver.switch_to.frame(iframe)

                # key in results and submit
                inputField = driver.find_element(By.ID, "audio-response")
                inputField.send_keys(key)
                delay()
                inputField.send_keys(Keys.ENTER)
                delay()

                err = driver.find_elements(By.CLASS_NAME, 'rc-audiochallenge-error-message')[0]
                if err.text == "" or err.value_of_css_property('display') == 'none':
                    print("[INFO] Success!")
                    break

        except Exception as e:
            print(e)
            sys.exit("[INFO] Possibly blocked by google. Change IP,Use Proxy method for requests")
    else:
        sys.exit("[INFO] Audio Play Button not found! In Very rare cases!")

options = webdriver.ChromeOptions() 
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.get('https://www.tnpublicnotice.com/')
driver.implicitly_wait(30)
#GUI needed
ROOT = tk.Tk()

ROOT.withdraw()
# the input dialog
starting_date_entry = simpledialog.askstring(title=county,
                                  prompt="Enter the starting date:")

# check it out
print(starting_date_entry)
ROOT = tk.Tk()

ROOT.withdraw()
# the input dialog
ending_date_entry = simpledialog.askstring(title=county,
                                  prompt="Enter the ending date:")

# check it out
print(ending_date_entry )
search_list=Select(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_ddlPopularSearches'))
search_list.select_by_visible_text('Probate Notices')
time.sleep(5)
driver.find_element(By.ID, "ctl00_ContentPlaceHolder1_as1_divCounty").click()
for i in driver.find_elements(By.XPATH, '//ul[@id="ctl00_ContentPlaceHolder1_as1_lstCounty"]/li'):
    if i.text.strip()==county.strip():
        ActionChains(driver).move_to_element(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_btnGo')).perform()
        print(i.text)
        i.click()
        ActionChains(driver).move_to_element(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_btnGo')).perform()
        break
time.sleep(2)
ActionChains(driver).move_to_element(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_btnGo')).perform()
driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_divDateRange').click()
driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_rbRange').click()

ActionChains(driver).move_to_element(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_txtDateFrom')).perform()
driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_txtDateFrom').clear()
driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_txtDateFrom').send_keys(starting_date_entry)
driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_txtDateTo').clear()
driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_txtDateTo').send_keys(ending_date_entry)
ActionChains(driver).move_to_element(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_btnGo')).perform()
driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_as1_btnGo').click()


time.sleep(5)
try:
    pages_viewable=Select(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_WSExtendedGridNP1_GridView1_ctl01_ddlPerPage'))
    pages_viewable.select_by_visible_text('50')
except:
    pass
time.sleep(2)

all_data=[]
try:
    total_pages=int(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_WSExtendedGridNP1_GridView1_ctl01_lblTotalPages').text.split()[1])
except:
    total_pages=1
for j in range(total_pages):
    for i in range(len(driver.find_elements(By.XPATH, "//td[@class='view bdrBrownTop bdrBrownRight bdrBrownBottom bdrBrownLeft mobileL mobileT']/input[1]"))):
        try:
            driver.find_elements(By.XPATH, f"//td[@class='view bdrBrownTop bdrBrownRight bdrBrownBottom bdrBrownLeft mobileL mobileT']/input[1]")[i].click()
        except IndexError as error:
            print('Done extraction')
            break
        while True:  
            try:
                if 'reCAPTCHA' in (driver.find_element(By.ID, "aspnetForm").text):
                    #ROOT = tk.Tk()
                    #ROOT.withdraw()
                    #USER_INP=simpledialog.askstring(title="Solve captcha manually",
                     #                     prompt="Have you done?:")
                    driver.find_element(By.XPATH, '//iframe[@title="reCAPTCHA"]').click()
                    captch_solve()
                    driver.switch_to.default_content()
                    driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_btnViewNotice').click()
                    print(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_pnlNoticeContent').text)
                    all_data.append(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_pnlNoticeContent').text)
                    driver.find_element(By.XPATH, '//p[@class="backlink"]/a').click()
                    driver.implicitly_wait(30)
                    break
                else:
                    print(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_pnlNoticeContent').text)
                    all_data.append(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_pnlNoticeContent').text)
                    driver.back()
                    driver.implicitly_wait(30)
                    break

            except:
                try:
                    driver.find_element(By.XPATH, '//iframe[@title="reCAPTCHA"]').click()
                    captch_solve()
                    driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_btnViewNotice').click()
                    print(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_pnlNoticeContent').text)
                    all_data.append(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_pnlNoticeContent').text)
                    driver.find_element(By.XPATH, '//p[@class="backlink"]/a').click()
                    driver.implicitly_wait(30)
                    break

                except:
                    driver.switch_to.default_content()
                    ROOT = tk.Tk()
                    ROOT.withdraw()
                    USER_INP=simpledialog.askstring(title="Solve captcha manually", prompt="Have you done? Also please click the View notice button")
                    print(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_pnlNoticeContent').text)
                    all_data.append(driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_PublicNoticeDetailsBody1_pnlNoticeContent').text)
                    driver.find_element(By.XPATH, '//p[@class="backlink"]/a').click()
                    driver.implicitly_wait(30)
                    break
    try:
        if total_pages>1:    
            driver.find_element(By.ID, 'ctl00_ContentPlaceHolder1_WSExtendedGridNP1_GridView1_ctl54_btnNext').click()
            driver.implicitly_wait(30)
    except:
        break

if county.strip()=='Robertson':    
    unmanaged_data=[]
    managed_data=[]
    other_county_data=[]

    count=1
    for i in all_data:
        extracted_text=i.splitlines()
        print(count)
        estate_of_status=False
        tn_status=False
        current_data=[]
        for j in extracted_text:
            if 'Estate of' in j: 
                estate_of_status=True
                estate_of=j.replace('Estate of','')
                estate_of=estate_of.replace(', Deceased','').strip()
                current_data.append(estate_of.strip())
                print('Estate of: ',estate_of)
            if ', TN' in j :
                if len(j)<90:
                    tn_status=True
                    city=j
                    print(city)
                    current_data.append(city.strip())
            if 'administratrix' in j.lower():
                if ',' in j:
                    executor=j.split(',')[0]
                else:
                    executor=extracted_text[extracted_text.index(j)-1]
                current_data.append(executor.strip())
            if 'executor' in j.lower():
                if ',' in j:
                    executor=j.split(',')[0]
                else:
                    executor=extracted_text[extracted_text.index(j)-1]
                current_data.append(executor.strip())
            if 'executrix' in j.lower():
                if ',' in j:
                    executor=j.split(',')[0]
                else:
                    executor=extracted_text[extracted_text.index(j)-1]
                current_data.append(executor.strip())
            if 'personal representative' in j.lower():
                if ',' in j:
                    executor=j.split(',')[0]
                else:
                    executor=extracted_text[extracted_text.index(j)-1]
                current_data.append(executor.strip())
            if 'administrator' in j.lower():
                if ',' in j:
                    executor=j.split(',')[0]
                else:
                    executor=extracted_text[extracted_text.index(j)-1]
                current_data.append(executor.strip())
            if 'attorney' in j.lower():
                if ',' in j:
                    attorney=j.split(',')[0]
                else:
                    attorney=extracted_text[extracted_text.index(j)-1]
                current_data.append(attorney.strip())
            if len(current_data)==4:
                break    
        if estate_of_status==False or tn_status==False:
            print('here')
            unmanaged_data.append(i)
        else:
            print(len(current_data))
            managed_data.append(current_data)
        count=count+1
    sumner_county_data=[]
    for i in managed_data:
        if 'Sumner' in i[0]:
            sumner_county_data.append(managed_data.pop(managed_data.index(i)))
    other_county_data.append(sumner_county_data)
    bold_font = Font(bold=True)
    bold_font = Font(bold=True)
    center_aligned_text = Alignment(horizontal="center")
    double_border_side = Side(border_style="double")
    square_border = Border(top=double_border_side,
                    right=double_border_side,
                    bottom=double_border_side,
                    left=double_border_side)
    workbook = Workbook()
    sheet = workbook.active
    sheet.title='Managed Data'
    sheet["A1"] = "County Name"
    sheet["A1"].font = bold_font
    sheet["A1"].alignment = center_aligned_text
    sheet["A1"].border = square_border
    sheet["B1"] = "City"
    sheet["B1"].font = bold_font
    sheet["B1"].alignment = center_aligned_text
    sheet["B1"].border = square_border
    sheet["C1"] = "Estate of"
    sheet["C1"].font = bold_font
    sheet["C1"].alignment = center_aligned_text
    sheet["C1"].border = square_border
    sheet["D1"] = "Executors"
    sheet["D1"].font = bold_font
    sheet["D1"].alignment = center_aligned_text
    sheet["D1"].border = square_border
    sheet["E1"] = "Attorney"
    sheet["E1"].font = bold_font
    sheet["E1"].alignment = center_aligned_text
    sheet["E1"].border = square_border

    dim_holder = DimensionHolder(worksheet=sheet)

    for col in range(sheet.min_column, sheet.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(sheet, min=col, max=col, width=20)

    sheet.column_dimensions = dim_holder
    for p,q in enumerate(managed_data):
        sheet[f"A{p+2}"]='Robertson'           
        sheet[f"B{p+2}"]=q[0]
        sheet[f"C{p+2}"]=q[1]
        sheet[f"D{p+2}"]=q[2]
        sheet[f"E{p+2}"]=q[3]
    #UNMANAGED DATA
    sheet=workbook.create_sheet("Sheet_A")
    sheet.title='Unmanaged Data'

    dim_holder = DimensionHolder(worksheet=sheet)

    for col in range(sheet.min_column, sheet.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(sheet, min=col, max=col, width=20)

    sheet.column_dimensions = dim_holder
    for p,q in enumerate(unmanaged_data):
        sheet[f"A{p+1}"]=q   
        
    #OTHER COUNTY DATA
    sheet=workbook.create_sheet("Sheet_B")
    sheet.title='Other County Data'
    sheet["A1"] = "County"
    sheet["A1"].font = bold_font
    sheet["A1"].alignment = center_aligned_text
    sheet["A1"].border = square_border
    sheet["B1"] = "City"
    sheet["B1"].font = bold_font
    sheet["B1"].alignment = center_aligned_text
    sheet["B1"].border = square_border
    sheet["C1"] = "Estate of"
    sheet["C1"].font = bold_font
    sheet["C1"].alignment = center_aligned_text
    sheet["C1"].border = square_border
    sheet["D1"] = "Executor"
    sheet["D1"].font = bold_font
    sheet["D1"].alignment = center_aligned_text
    sheet["D1"].border = square_border
    sheet["E1"] = "Attorney"
    sheet["E1"].font = bold_font
    sheet["E1"].alignment = center_aligned_text
    sheet["E1"].border = square_border

    dim_holder = DimensionHolder(worksheet=sheet)

    for col in range(sheet.min_column, sheet.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(sheet, min=col, max=col, width=20)

    sheet.column_dimensions = dim_holder
    for p,q in enumerate(other_county_data[0]):
        sheet[f"A{p+2}"]='Sumner'
        sheet[f"B{p+2}"]=q[0]           
        sheet[f"C{p+2}"]=q[1]           
        sheet[f"D{p+2}"]=q[2]
        sheet[f"E{p+2}"]=q[3]

    workbook.save(f"tnpublicnotices_Probates_{county}_[{starting_date_entry.replace('/','-')}-{ending_date_entry.replace('/','-')}].xlsx")
if county.strip()=='Wilson':
    managed_data=[]
    for i in all_data:
        extracted_text=i.splitlines()
        estate_of_found=True
        executor='-'
        attorney='-'
        for j in extracted_text:
                if ('estate of' in j.lower()) and (estate_of_found):
                    es=j.replace(':','')
                    es=es.title()
                    estate_of_extracter=es.split('Of')
                    if estate_of_extracter[-1]=='':
                        estate_of=extracted_text[extracted_text.index(j)+1].title()
                        estate_of_found=False
                    if estate_of_extracter[-1]!='':
                        estate_of=es.split('Of')[-1].strip()
                        estate_of_found=False
                if (', deceased' in j.lower()) and (estate_of_found):
                        estate_of=j.split(',')[0].title()
                        estate_of_found=False
                if ('of Property:' in j) and (estate_of_found):
                    estate_of=j.split(':')[-1].strip().title()
                    estate_of_found=False
                if ('OWNER(S):' in j) and (estate_of_found):
                    estate_of=j.split(':')[-1].strip().title()
                    estate_of_found=False 
                if ('PERSONAL' in j.upper()) or ('executor' in j.lower()) or ('administratrix' in j.lower()) or('executrix' in j.lower()) or('administrator' in j.lower()):
                    executor=extracted_text[extracted_text.index(j)-1]
                if 'Attorney' in j or 'ATTORNEY' in j:
                    if ',' in j:
                        attorney=j.split(',')[0]
                    else:
                        attorney=extracted_text[extracted_text.index(j)-1]


        #print('Estate of: ',estate_of.strip())
        #print('Executor: ',executor.title())
        #print('Attorney: ',attorney.title().strip())
        #print('****************************************')
        managed_data.append([estate_of.strip(),executor.title(),attorney.title().strip()])
    bold_font = Font(bold=True)
    bold_font = Font(bold=True)
    center_aligned_text = Alignment(horizontal="center")
    double_border_side = Side(border_style="double")
    square_border = Border(top=double_border_side,
                    right=double_border_side,
                    bottom=double_border_side,
                    left=double_border_side)
    workbook = Workbook()
    sheet = workbook.active
    sheet.title='Managed Data'
    sheet["A1"] = "County Name"
    sheet["A1"].font = bold_font
    sheet["A1"].alignment = center_aligned_text
    sheet["A1"].border = square_border
    sheet["B1"] = "Estate of"
    sheet["B1"].font = bold_font
    sheet["B1"].alignment = center_aligned_text
    sheet["B1"].border = square_border
    sheet["C1"] = "Executors"
    sheet["C1"].font = bold_font
    sheet["C1"].alignment = center_aligned_text
    sheet["C1"].border = square_border
    sheet["D1"] = "Attorney"
    sheet["D1"].font = bold_font
    sheet["D1"].alignment = center_aligned_text
    sheet["D1"].border = square_border

    dim_holder = DimensionHolder(worksheet=sheet)

    for col in range(sheet.min_column, sheet.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(sheet, min=col, max=col, width=20)

    sheet.column_dimensions = dim_holder
    for p,q in enumerate(managed_data):
        sheet[f"A{p+2}"]='Wilson'           
        sheet[f"B{p+2}"]=q[0]
        sheet[f"C{p+2}"]=q[1]
        sheet[f"D{p+2}"]=q[2]
    workbook.save(f"tnpublicnotices_Probates_{county}_[{starting_date_entry.replace('/','-')}-{ending_date_entry.replace('/','-')}].xlsx")
if county.strip()=='Williamson':
    pass
if county.strip()=='Rutherford':
    trustee_sale_data=[]
    substitute_trustee_data=[]
    foreclosure_sale_data=[]
    unfilterable_data=[]
    notice_to_creitors_data=[]
    for i in all_data:
        executed_by='Not available'
        property_address='Not available'
        interested_parties='Not available'
        no_of_parties='Not available'
        property_owners='Not available'
        interested_parties='Not available'
        if 'NOTICE OF TRUSTEE\'S SALE' in i:
            print('NOTICE OF TRUSTEE\'S SALE')
            extracted_text=i.splitlines()
            for j in extracted_text:
                if 'executed by' in j:
                    executed_by=((j[j.index('executed by'):])[:(j[j.index('executed by'):]).index(',')].replace('executed by','').strip())
                    
                if 'ALSO KNOWN AS' in j:
                    property_address=(j.replace('ALSO KNOWN AS:','').strip())
                    
                if ('The sale held pursuant to this Notice' in j):
                    count=1
                    factor=1
                    if 'On or about' in extracted_text[extracted_text.index(j)-1]:
                        count=2
                        factor=2
                    interested_parties=''
                    while True:
                        interested_parties+=(extracted_text[extracted_text.index(j)-count])+'|'
                        count=count+1
                        if 'referenced property:' in extracted_text[extracted_text.index(j)-count]:
                            
                            no_of_parties=count-factor
                            
                            break
            print('Executed by: ',executed_by)
            print('Address: ',property_address)
            print('Interested parties: ',interested_parties)
            print('Number of parties: ',no_of_parties)
            trustee_sale_data.append([executed_by,property_address,interested_parties,no_of_parties])
        elif ('SUBSTITUTE TRUSTEE\'S SALE' in i) or ('SUBSTITUTE TRUSTEES SALE' in i) or ('SUBSTITUTE TRUSTEE\'S NOTICE OF SALE' in i):
            print('SUBSTITUTE TRUSTEE')
            extracted_text=i.splitlines()
            for j in extracted_text:
                if 'executed by' in j:
                    executed_by=((j[j.index('executed by'):])[:(j[j.index('executed by'):]).index(',')].replace('executed by','').strip())
                if ('of Property:' in j) or ('OWNER(S):' in j):
                    property_owners=j[j.index(':'):].replace(':','').strip()                  
                if 'street address' in j.lower():
                    if ':' in j:
                        property_address=j[j.index(':'):].replace(':','').strip()
                    if 'In the event of' in j:
                        print('here')
                        property_address=((j[j.index('is believed to be'):])[:(j[j.index('is believed to be'):]).index('.')].replace('is believed to be','').strip())               
                    if 'but such address' in j:
                        property_address=((j[j.index('is believed to be'):])[:(j[j.index('is believed to be'):]).index(', but such address')].replace('is believed to be','').strip())               
                        print(property_address)
                if 'OTHER INTERESTED PARTIES:' in j.upper():
                    interested_parties=j[j.index(':'):].replace(':','').strip()
                    if interested_parties=='':
                        count=1
                        interested_parties=''
                        while True:
                            interested_parties+=(extracted_text[extracted_text.index(j)+count])+'|'
                            count=count+1
                            if 'The sale of the above-described property' in extracted_text[extracted_text.index(j)+count]:
                                break
                    if 'AM, local time,' in interested_parties:
                        interested_parties=(j[j.index('Other interested parties'):j.index('The hereinafter described')]).replace('The hereinafter described','')
                        interested_parties=interested_parties.replace('Other interested parties','')
                        interested_parties=interested_parties.replace(':','')
                        

            if property_owners=='Not available':
                property_owners=executed_by
            print('Property owner: ',property_owners)
            print('Address: ',property_address)
            print('Interested parties: ',interested_parties)
            substitute_trustee_data.append([property_owners,property_address,interested_parties])
        elif 'NOTICE OF SUBSTITUTE TRUSTEE`S SALE' in i:
            print('NOTICE OF SUBSTITUTE TRUSTEE`S SALE 2nd kind')
            extracted_text=(nltk.tokenize.sent_tokenize(i))
            for j in extracted_text:
                if 'executed by' in j:
                    executed_by=((j[j.index('executed by'):])[:(j[j.index('executed by'):]).index(',')].replace('executed by','').strip())     
                if 'street address' in j.lower():
                    property_address=((j[j.index('Commonly known as'):])[:(j[j.index('Commonly known as'):]).index('The street address')].replace('Commonly known as','').strip())   
                if 'referenced property:' in j.lower():
                    interested_parties=((j[j.index('referenced property:'):])[:(j[j.index('referenced property:'):]).index('.')].replace('referenced property:','').strip())    
            print('Property owner: ',executed_by)
            print('Address: ',property_address)
            print('Interested parties: ',interested_parties)
            substitute_trustee_data.append([executed_by,property_address,interested_parties])
        elif 'NOTICE OF FORECLOSURE SALE' in i.upper():
            print('NOTICE OF FORECLOSURE SALE')
            extracted_text=i.splitlines()
            for j in extracted_text:
                if 'owner(s)' in j.lower():
                    property_owners=j[j.index(':'):].replace(':','').strip() 
                elif 'TRUSTEE' in j:
                    property_owners=extracted_text[extracted_text.index(j)+1].strip()
                if ('address' in j.lower()) and (':' in j):
                    property_address=j[j.index(':'):].replace(':','').strip()
                elif ('address' in j.lower() and 'real estate' in j.lower()):
                    property_address=j[j.index('notice is'):j.index('.')].replace('notice is','').strip()
                if (('interested' in j.lower()) and ('party(ies)' in j.lower()) and (':' in j)) or (('interested' in j.lower()) and ('parties' in j.lower()) and (':' in j)): 
                    interested_parties=j[j.index(':'):].replace(':','').strip()
                    
            print('Property owner: ',property_owners)
            print('Address: ',property_address)
            print('Interested parties: ',interested_parties)
            foreclosure_sale_data.append([property_owners,property_address,interested_parties])
            
        elif 'Notice to Creditors' in i:
            extracted_text=i.splitlines()
            for j in extracted_text:
                if ('Estate of' in j) and len(j)>60:
                    if 'Deceased' in j:
                        property_owners=((j[j.index('Estate of'):])[:(j[j.index('Estate of'):]).index('Deceased')]).replace('Estate of','').strip()
                        property_owners.replace(',','')
                if ('Estate of' in j) and len(j)<=60:
                    property_owners=(j.replace('Estate of','')).strip()
                    property_owners.replace(',','')
                if 'executor' in j.lower():
                    executed_by=(extracted_text[extracted_text.index(j)-1])
                if 'administrator' in j.lower():
                    executed_by=(extracted_text[extracted_text.index(j)-1])
                if 'personal representative' in j.lower():
                    executed_by=(extracted_text[extracted_text.index(j)-1])
                if 'attorney' in j.lower():
                    attorney=(extracted_text[extracted_text.index(j)-1])
            if ',' in property_owners:
                property_owners=property_owners[:-1]
            print('Property owner:',property_owners)
            print('Executors:',executed_by)
            print('Attorney:',attorney)
            notice_to_creitors_data.append([property_owners,executed_by,attorney])
        else:
            print('unfiltered')
            unfilterable_data.append(i)
        print('**************************************')
    trustee_sale_data_excel=[["County", "Property Owners", "Property Address", "Interested Parties"]]
    substitute_trustee_data_excel=[["County", "Property Owners", "Property Address", "Interested Parties"]]
    foreclosure_sale_data_excel=[["County", "Property Owners", "Property Address", "Interested Parties"]]
    unfilterable_data_excel=[]
    notice_to_creitors_data_excel=[["County", "Property Owners", "Property Address", "Interested Parties"]]
    for i in trustee_sale_data:
        if i not in trustee_sale_data_excel:
            trustee_sale_data_excel.append(i)
    for i in substitute_trustee_data:
        if i not in substitute_trustee_data_excel:
            substitute_trustee_data_excel.append(i)
    for i in foreclosure_sale_data:
        if i not in foreclosure_sale_data_excel:
            foreclosure_sale_data_excel.append(i)
    for i in unfilterable_data:
        if i not in unfilterable_data_excel:
            unfilterable_data_excel.append(i)
    for i in notice_to_creitors_data:
        if i not in notice_to_creitors_data_excel:
            notice_to_creitors_data_excel.append(i)
    # print(len(trustee_sale_data))
    # print(len(substitute_trustee_data))
    # print(len(foreclosure_sale_data))
    # print(len(unfilterable_data))
    # print(len(trustee_sale_data_excel))
    # print(len(substitute_trustee_data_excel))
    # print(len(foreclosure_sale_data_excel))
    # print(len(unfilterable_data_excel))

    
    ##############################################################################################################
    
    bold_font = Font(bold=True)
    bold_font = Font(bold=True)
    center_aligned_text = Alignment(horizontal="center")
    double_border_side = Side(border_style="double")
    square_border = Border(top=double_border_side,
                    right=double_border_side,
                    bottom=double_border_side,
                    left=double_border_side)
    workbook = Workbook()
    sheet = workbook.active
    
    #Notice to creditors
    sheet.title='Notice to Creditors'
    sheet["A1"] = "County Name"
    sheet["A1"].font = bold_font
    sheet["A1"].alignment = center_aligned_text
    sheet["A1"].border = square_border
    sheet["B1"] = "Estate of"
    sheet["B1"].font = bold_font
    sheet["B1"].alignment = center_aligned_text
    sheet["B1"].border = square_border
    sheet["C1"] = "Executors"
    sheet["C1"].font = bold_font
    sheet["C1"].alignment = center_aligned_text
    sheet["C1"].border = square_border
    sheet["D1"] = "Attorney"
    sheet["D1"].font = bold_font
    sheet["D1"].alignment = center_aligned_text
    sheet["D1"].border = square_border
    dim_holder = DimensionHolder(worksheet=sheet)

    for col in range(sheet.min_column, sheet.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(sheet, min=col, max=col, width=20)

    sheet.column_dimensions = dim_holder
    for p,q in enumerate(notice_to_creitors_data_excel):
        sheet[f"A{p+2}"]='Rutherford'           
        sheet[f"B{p+2}"]=q[0]
        sheet[f"C{p+2}"]=q[1]
        sheet[f"D{p+2}"]=q[2]
        
    #trustee_sale_data
    sheet=workbook.create_sheet("Sheet_A")
    sheet.title='Trustee Sale Data'
    sheet["A1"] = "County"
    sheet["A1"].font = bold_font
    sheet["A1"].alignment = center_aligned_text
    sheet["A1"].border = square_border
    sheet["B1"] = "Executors"
    sheet["B1"].font = bold_font
    sheet["B1"].alignment = center_aligned_text
    sheet["B1"].border = square_border
    sheet["C1"] = "Property Address"
    sheet["C1"].font = bold_font
    sheet["C1"].alignment = center_aligned_text
    sheet["C1"].border = square_border
    sheet["D1"] = "Interested Parties"
    sheet["D1"].font = bold_font
    sheet["D1"].alignment = center_aligned_text
    sheet["D1"].border = square_border
    sheet["E1"] = "No. of Interested Parties"
    sheet["E1"].font = bold_font
    sheet["E1"].alignment = center_aligned_text
    sheet["E1"].border = square_border

    dim_holder = DimensionHolder(worksheet=sheet)

    for col in range(sheet.min_column, sheet.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(sheet, min=col, max=col, width=20)

    sheet.column_dimensions = dim_holder
    for p,q in enumerate(trustee_sale_data_excel):
        sheet[f"A{p+2}"]='Rutherford'           
        sheet[f"B{p+2}"]=q[0]
        sheet[f"C{p+2}"]=q[1]
        sheet[f"D{p+2}"]=q[2]
        sheet[f"E{p+2}"]=q[3]
        
    #substitute_trustee_data
    sheet=workbook.create_sheet("Sheet_B")
    sheet.title='Substitute Trustee Data'
    sheet["A1"] = "County"
    sheet["A1"].font = bold_font
    sheet["A1"].alignment = center_aligned_text
    sheet["A1"].border = square_border
    sheet["B1"] = "Property Owners"
    sheet["B1"].font = bold_font
    sheet["B1"].alignment = center_aligned_text
    sheet["B1"].border = square_border
    sheet["C1"] = "Property Address"
    sheet["C1"].font = bold_font
    sheet["C1"].alignment = center_aligned_text
    sheet["C1"].border = square_border
    sheet["D1"] = "Interested Parties"
    sheet["D1"].font = bold_font
    sheet["D1"].alignment = center_aligned_text
    sheet["D1"].border = square_border
    dim_holder = DimensionHolder(worksheet=sheet)

    for col in range(sheet.min_column, sheet.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(sheet, min=col, max=col, width=20)

    sheet.column_dimensions = dim_holder
    for p,q in enumerate(substitute_trustee_data_excel):
        sheet[f"A{p+2}"]='Rutherford'
        sheet[f"B{p+2}"]=q[0]           
        sheet[f"C{p+2}"]=q[1]           
        sheet[f"D{p+2}"]=q[2]

    #foreclosure_sale_data
    sheet=workbook.create_sheet("Sheet_C")
    sheet.title='Foreclosure Sale Data'
    sheet["A1"] = "County"
    sheet["A1"].font = bold_font
    sheet["A1"].alignment = center_aligned_text
    sheet["A1"].border = square_border
    sheet["B1"] = "Property Owners"
    sheet["B1"].font = bold_font
    sheet["B1"].alignment = center_aligned_text
    sheet["B1"].border = square_border
    sheet["C1"] = "Property Address"
    sheet["C1"].font = bold_font
    sheet["C1"].alignment = center_aligned_text
    sheet["C1"].border = square_border
    sheet["D1"] = "Interested Parties"
    sheet["D1"].font = bold_font
    sheet["D1"].alignment = center_aligned_text
    sheet["D1"].border = square_border
    dim_holder = DimensionHolder(worksheet=sheet)

    for col in range(sheet.min_column, sheet.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(sheet, min=col, max=col, width=20)

    sheet.column_dimensions = dim_holder
    for p,q in enumerate(foreclosure_sale_data_excel):
        sheet[f"A{p+2}"]='Rutherford'
        sheet[f"B{p+2}"]=q[0]           
        sheet[f"C{p+2}"]=q[1]           
        sheet[f"D{p+2}"]=q[2]
        
    #unfilterable data
    sheet=workbook.create_sheet("Sheet_D")
    sheet.title='Unfilterable Data'

    dim_holder = DimensionHolder(worksheet=sheet)

    for col in range(sheet.min_column, sheet.max_column + 1):
        dim_holder[get_column_letter(col)] = ColumnDimension(sheet, min=col, max=col, width=20)

    sheet.column_dimensions = dim_holder
    for p,q in enumerate(unfilterable_data_excel):
        sheet[f"A{p+1}"]=q           

    workbook.save(f"tnpublicnotices_{county}_[{starting_date_entry.replace('/','-')}-{ending_date_entry.replace('/','-')}].xlsx")

    ###################################################################################################################
    
    SERVICE_ACCOUNT_FILE = 'keys.json'
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

    creds = None
    creds = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    SAMPLE_SPREADSHEET_ID = '1278HJam-TruXlKrlHA3mdTHtUEljeK-xEC4co0kytaA'


    try:
        service = build('sheets', 'v4', credentials=creds)

        sheet = service.spreadsheets()

        request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range="Notice to Creditors!A1", valueInputOption="USER_ENTERED", body={"values":notice_to_creitors_data_excel}).execute()

        request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range="Trustee Sale Data!A1", valueInputOption="USER_ENTERED", body={"values":trustee_sale_data_excel}).execute()

        request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range="Substitute Trustee Data!A1", valueInputOption="USER_ENTERED", body={"values":substitute_trustee_data_excel}).execute()

        request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range="Foreclosure Sale Data!A1", valueInputOption="USER_ENTERED", body={"values":foreclosure_sale_data_excel}).execute()

        request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range="Unfilterable Data!A1", valueInputOption="USER_ENTERED", body={"values":unfilterable_data_excel}).execute()

    except HttpError as err:
        print(err)
    
    ###################################################################################################################
driver.close()

2/7/2022
7/7/2022
Williamson
[INFO] Audio src: https://www.google.com/recaptcha/api2/payload?p=06AIIukzifwvPjbq3_PpoQ98jxxwTx7ueMz6bcjKp0NXFPFbArZlRE-uUbe3snwNwvGWqQbGjRjAQ1VHYBVMTN09qBhqB9UNAehrRPAN4GnOj-uvVv5OeQvrJ2_7IoeGvbzTTbVxcsCZbtxDSb44HJp_YFW7vCPWKyoQMwuMvhs0EiqJlQHUBLlTuGsjpjXvV9BLPjGR2qS6_v&k=6LeK42oUAAAAAFG3gQ8C4gk9wYrIYptUDxNO4D5M
[INFO] Recaptcha Key: Trunk interface which.
[INFO] Success!
Notice Content


NOTICE TO CREDITORS T.C.A. § 30-2-306 CHANCERY COURT FOR WILLIAMSON COUNTY, TENNESSEE P. O. Box 1666 Franklin, TN 37065 Estate of: Charles Reid McPherson Docket No. 22PR-12442 Notice is hereby given that on the 22nd day of June, 2022, Letters Testamentary in respect of the Estate of Charles Reid McPherson deceased, who died on May 14, 2022, were issued to the undersigned by the Chancery Court of Williamson County, Tennessee. All persons, resident and non-resident, having claims, matured or unmatured, against the Estate are required to file the same with the Clerk of the 

Notice Content


NOTICE TO CREDITORS T.C.A. § 30-2-306 CHANCERY COURT FOR WILLIAMSON COUNTY, TENNESSEE P. O. Box 1666 Franklin, TN 37065 Estate of: Virginia Houser Docket No. 22PR-12473 Notice is hereby given that on the 29th day of June, 2022, Letters Testamentary in respect of the Estate of Virginia Houser, deceased, who died on April 27, 2022, were issued to the undersigned by the Chancery Court of Williamson County, Tennessee. All persons, resident and non-resident, having claims, matured or unmatured, against the Estate are required to file the same with the Clerk of the above-named Court on or before the earlier of the dates prescribed in (1) or (2) otherwise their claims will be forever barred: (1) (A) Four (4) months from the date of the first publication (or the posting, as the case may be) of this notice if the creditor received an actual copy of this notice to creditors at least sixty (60) days before the date that is four (4) months from the date of the first publication (o

Notice Content


NOTICE TO CREDITORS T.C.A. § 30-2-306 CHANCERY COURT FOR WILLIAMSON COUNTY, TENNESSEE P. O. Box 1666 Franklin, TN 37065 Estate of: Timothy L. Coomer Docket No. 22PR-12478 Notice is hereby given that on the 29th day of June, 2022, Letters Testamentary in respect of the Estate of Timothy L. Coomer, deceased, who died on May 4, 2022, were issued to the undersigned by the Chancery Court of Williamson County, Tennessee. All persons, resident and non-resident, having claims, matured or unmatured, against the Estate are required to file the same with the Clerk of the above-named Court on or before the earlier of the dates prescribed in (1) or (2) otherwise their claims will be forever barred: (1) (A) Four (4) months from the date of the first publication (or the posting, as the case may be) of this notice if the creditor received an actual copy of this notice to creditors at least sixty (60) days before the date that is four (4) months from the date of the first publication (

Notice Content


0005305525 SUCCESSOR TRUSTEE'S NOTICE OF SALE WHEREAS, by Deed of Trust dated the 11th day of April, 2012, of record in Book 5659, Page 361, in the Register's Office for Williamson County, TN, CHARLES F. BYRD, JR. and ANNETTE H. BYRD who acquired title as EVELYN A. BYRD (the "Grantors") conveyed to Amelia A. Shreve, Trustee, the hereinafter described real estate to secure the payment of Grantors' Promissory Note owing to FIRST HORIZON BANK, a Tennessee banking corporation, successor by conversion to First Tennessee Bank National Association, a national banking association (the "Beneficiary"); and WHEREAS, GORDON D. FOSTER was appointed Successor Trustee of said Deed of Trust by the Beneficiary, said Appointment being recorded in the aforesaid Register's Office; and WHEREAS, default has occurred and I have been directed to foreclose said Deed of Trust and sell said real estate. NOW, THEREFORE, by virtue of the authority vested in me by said Deed of Trust, I will offer f

[INFO] Audio src: https://www.google.com/recaptcha/api2/payload?p=06AIIukzi-cUWPOrjHwtNwxjUzHAoF1Ej5kZxpfiiDrrJFT71AKXtB4lzNYpD9JAAze1a845hYBI7U6CKJPDWCxXqaRJYxFJWUfzRmUefp9swIuQdVGNH-aIDult86eG4fp8rFNegR2PEuk-TYENks1JZqs7plwvgpnTP1pQufcUUrxcXvDAgxHR3ezuzue2cPYnQqY2xMwmORN_TAgZiiupfSzdVSl61OTg&k=6LeK42oUAAAAAFG3gQ8C4gk9wYrIYptUDxNO4D5M
[INFO] Recaptcha Key: Attackers could potentially act.
[INFO] Success!
Notice Content


NOTICE OF TRUSTEE'S SALE

WHEREAS, default has occurred in the performance of the covenants, terms, and conditions of a Deed of Trust Note dated April 20, 2018, and the Deed of Trust of even date securing the same, recorded April 23, 2018, in Book No. 7343, at Page 844, and modified on October 5, 2021, In Book No. 8754, At Page 407 in Office of the Register of Deeds for Williamson County, Tennessee, executed by Ho Tran, conveying certain property therein described to First American Title as Trustee for Mortgage Electronic Registration Systems, Inc., as beneficiary, as

...

Notice Content


NOTICE OF TRUSTEE'S SALE

WHEREAS, default has occurred in the performance of the covenants, terms, and conditions of a Deed of Trust Note dated June 2, 2003, and the Deed of Trust of even date securing the same, recorded June 2, 2003, in Book No. 2864, at Page 194, in Office of the Register of Deeds for Williamson County, Tennessee, executed by Jeanette L Herdman and Robert V Herdman, conveying certain property therein described to Premier Escrow, LLC as Trustee for Fifth Third Mortgage Company; and the undersigned, Wilson & Associates, P.L.L.C., having been appointed Successor Trustee by Fifth Third Bank, National Association.
NOW, THEREFORE, notice is hereby given that the entire indebtedness has been declared due and payable; and that an agent of Wilson & Associates, P.L.L.C., as Successor Trustee, by virtue of the power, duty, and authority vested in and imposed upon said Successor Trustee, by Fifth Third Bank, National Association, will, on June 15, 2022 on or ab

Notice Content


City of Franklin Meeting Agenda 109 3rd Ave S Franklin, TN 37064 (615) 791-3217 Franklin Municipal Planning Commission Thursday, May 26, 2022 7:00 PM Board Room Notice is hereby given that a meeting of the Franklin Municipal Planning Commission will be held on the date, time and at the location listed above. Additional information can be found at www.franklintn.gov/planning. The typical process for discussing an item is as follows: 1. Staff Presentation 2. Public comments 3. Applicant presentation, and 4. Motion / discussion / vote Applicants are encouraged to attend the meeting, even if they agree with the staff recommendation. The Planning Commission may defer or disapprove an application/request unless someone is present to represent it. For accommodations due to disabilities or other special arrangements, please contact the Human Resources Department at (615) 791-3216, at least 24 hours prior to the meeting. Accommodations have been made to ensure that the public i

Notice Content


NOTICE OF TRUSTEE'S SALE

WHEREAS, default has occurred in the performance of the covenants, terms, and conditions of a Deed of Trust Note dated June 2, 2003, and the Deed of Trust of even date securing the same, recorded June 2, 2003, in Book No. 2864, at Page 194, in Office of the Register of Deeds for Williamson County, Tennessee, executed by Jeanette L Herdman and Robert V Herdman, conveying certain property therein described to Premier Escrow, LLC as Trustee for Fifth Third Mortgage Company; and the undersigned, Wilson & Associates, P.L.L.C., having been appointed Successor Trustee by Fifth Third Bank, National Association.
NOW, THEREFORE, notice is hereby given that the entire indebtedness has been declared due and payable; and that an agent of Wilson & Associates, P.L.L.C., as Successor Trustee, by virtue of the power, duty, and authority vested in and imposed upon said Successor Trustee, by Fifth Third Bank, National Association, will, on June 15, 2022 on or ab

Notice Content


NOTICE TO CREDITORS T.C.A. § 30-2-306 CHANCERY COURT FOR WILLIAMSON COUNTY, TENNESSEE P. O. Box 1666 Franklin, TN 37065 Estate of: Nichole Ridnouer-Tenllado Docket No. 22PR-12334 Notice is hereby given that on the 27th day of April, 2022, Letters Testamentary in respect of the Estate of Nichole Ridnouer-Tenllado, deceased, who died on Febrary 4, 2022, were issued to the undersigned by the Chancery Court of Williamson County, Tennessee. All persons, resident and non-resident, having claims, matured or unmatured, against the Estate are required to file the same with the Clerk of the above-named Court on or before the earlier of the dates prescribed in (1) or (2) otherwise their claims will be forever barred: (1) (A) Four (4) months from the date of the first publication (or the posting, as the case may be) of this notice if the creditor received an actual copy of this notice to creditors at least sixty (60) days before the date that is four (4) months from the date of th

[INFO] Recaptcha Key: The provision of maintenance but.
[INFO] Success!
Notice Content


NOTICE TO CREDITORS T.C.A. § 30-2-306 CHANCERY COURT FOR WILLIAMSON COUNTY, TENNESSEE P. O. Box 1666 Franklin, TN 37065 Estate of: Frank Weiland Stubblefield Docket No. 22PR-12358 Notice is hereby given that on the 29th day of April, 2022, Letters of Administration in respect of the Estate of Frank Weiland Stubblefield, deceased, who died on May 24, 2021, were issued to the undersigned by the Chancery Court of Williamson County, Tennessee. All persons, resident and non-resident, having claims, matured or unmatured, against the Estate are required to file the same with the Clerk of the above-named Court on or before the earlier of the dates prescribed in (1) or (2) otherwise their claims will be forever barred: (1) (A) Four (4) months from the date of the first publication (or the posting, as the case may be) of this notice if the creditor received an actual copy of this notice to creditors at least

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

SERVICE_ACCOUNT_FILE = 'keys.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

SAMPLE_SPREADSHEET_ID = '1278HJam-TruXlKrlHA3mdTHtUEljeK-xEC4co0kytaA'


try:
    service = build('sheets', 'v4', credentials=creds)
                                                      
    sheet = service.spreadsheets()
    
    request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="Notice to Creditors!A1", valueInputOption="USER_ENTERED", body={"values":notice_to_creitors_data_excel}).execute()
    
    request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="Trustee Sale Data!A1", valueInputOption="USER_ENTERED", body={"values":trustee_sale_data_excel}).execute()
    
    request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="Substitute Trustee Data!A1", valueInputOption="USER_ENTERED", body={"values":substitute_trustee_data_excel}).execute()
    
    request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="Foreclosure Sale Data!A1", valueInputOption="USER_ENTERED", body={"values":foreclosure_sale_data_excel}).execute()
    
    request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="Unfilterable Data!A1", valueInputOption="USER_ENTERED", body={"values":unfilterable_data_excel}).execute()
                                                      
except HttpError as err:
    print(err)

In [6]:
substitute_trustee_data_excel

NameError: name 'substitute_trustee_data_excel' is not defined